In [9]:
import datetime
import ast
import json
from collections import Counter

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

In [10]:
df = pd.read_csv('new_cars.csv')
pd.set_option('display.max_columns', None)
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,transmission,engine,vin,stock_number,vehicle_history,seller_name,price_history,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,low mpg,high mpg
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,79497,17899,Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',...",None reported,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,NaN,21.0,27.0
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,80326,27900,This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,NaN,16.0,23.0
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,92959,18950,This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,black,31.0,36.0
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,59176,39950,Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',...",At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,NaN,19.0,24.0
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,135385,26950,This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,-,3.6L V6 24V MPFI DOHC,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',...",None reported,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,NaN,16.0,21.0


In [11]:
### Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'price_history', 'vehicle_history', 'listing_title', 'interior_color']
# add interior_color back once normalized
df = df.drop(columns=columns_to_drop)

df.head()

,listing_mileage,primary_price,drivetrain,transmission,engine,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,2.4L I4 16V GDI DOHC,ETHIO Motors,None reported,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,NaN,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,5.7L V8 16V MPFI OHV,Custom Cars West,None reported,No,Yes,NaN,NaN,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,NaN,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,1.6L I4 16V MPFI DOHC,Custom Cars West,None reported,No,Yes,NaN,NaN,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,black,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,5.7L V8 16V MPFI OHV,Custom Cars West,At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,NaN,19.0,24.0
4,135385,26950,Four-wheel Drive,-,3.6L V6 24V MPFI DOHC,Custom Cars West,None reported,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,NaN,16.0,21.0
